# Genre Classification 2

In [1]:
import pandas as pd
import numpy as np
import sklearn.preprocessing as pp

In [2]:
#read data from .pkl object file
df = pd.read_pickle('labels_features.pkl')
df.head()

,track_id,zcr_mean,zcr_var,spec_cent_mean,spec_cent_var,spec_roll_mean,spec_roll_var,mfccs_mean,mfccs_var,genre_top
0,2,0.163695,0.008725,3037.644725,9.809958e+05,6214.647549,2.281126e+06,"[-67.3842, 65.155075, -10.669196, 12.176932, -...","[2907.9536, 927.7224, 519.6038, 312.80936, 306...",Hip-Hop
1,5,0.100023,0.004433,2414.124259,7.569450e+05,5437.441882,2.535670e+06,"[-106.58773, 87.23936, 12.156827, 24.140398, -...","[2197.3853, 843.6833, 457.25446, 314.9391, 276...",Hip-Hop
2,10,0.148674,0.000798,2349.559292,1.419561e+05,4315.856035,6.900174e+05,"[-17.238174, 94.44203, -48.614548, 32.370766, ...","[797.04956, 417.18228, 307.51672, 135.75954, 1...",Pop
3,140,0.044322,0.002694,1746.322280,1.049483e+06,3863.175065,5.928252e+06,"[-278.57538, 101.53074, 36.42724, 35.59012, 26...","[7312.72, 1265.0995, 471.97086, 237.86455, 195...",Folk
4,141,0.061836,0.002676,1654.237111,6.403904e+05,3315.637916,2.762245e+06,"[-208.56926, 116.633934, -1.7237115, 26.018599...","[12825.669, 1198.2709, 278.61575, 253.6174, 20...",Folk


## Build Feature Vector

In [3]:
#combine all features into one vector for training
mfccs_mean = np.nan_to_num(np.stack(df['mfccs_mean']))
x = mfccs_mean

mfccs_var = np.nan_to_num(np.stack(df['mfccs_var']))
x = np.append(x,mfccs_var,1)

zcr_mean = np.zeros((len(x),1))
zcr_mean[:,0] = np.array(df['zcr_mean'])
x = np.append(x,zcr_mean,1)

zcr_var = np.zeros((len(x),1))
zcr_var[:,0] = np.array(df['zcr_var'])
x = np.append(x,zcr_var,1)

sc_mean = np.zeros((len(x),1))
sc_mean[:,0] = np.array(df['spec_cent_mean'])
x = np.append(x,sc_mean,1)

sc_var = np.zeros((len(x),1))
sc_var[:,0] = np.array(df['spec_cent_var'])
x = np.append(x,sc_var,1)

sr_mean = np.zeros((len(x),1))
sr_mean[:,0] = np.array(df['spec_roll_mean'])
x = np.append(x,sr_mean,1)

sr_var = np.zeros((len(x),1))
sr_var[:,0] = np.array(df['spec_roll_var'])
x = np.append(x,sr_var,1)

x = pp.scale(x) #scale features to zero mean unit variance

In [48]:
display(mfccs_mean.shape)

(7997, 20)

## Preprocess Labels and Split Train/Test

In [4]:
from sklearn.model_selection import train_test_split
from ast import literal_eval

y_lab = np.array(df['genre_top'],dtype=str) #labels

#encode labels as numbers
le = pp.LabelEncoder()
le.fit(y_lab)
y = le.transform(y_lab)

#train/test split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

## ML Models

### Support Vector Machine

In [5]:
from sklearn import svm

clf = svm.SVC()
clf.fit(x_train, y_train)
clf.score(x_test,y_test)

0.5025

### Random Forest

In [6]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier()
rf.fit(x_train, y_train)
rf.score(x_test,y_test)

0.484375

### K-Nearest Neighbors

In [10]:
from sklearn.neighbors import KNeighborsClassifier

kn = KNeighborsClassifier()
kn.fit(x_train, y_train)
kn.score(x_test, y_test)

0.45

### CNN

In [12]:
from keras import models
from keras import layers

model = models.Sequential()
model.add(layers.Dense(256, activation='relu', input_shape=(x_train.shape[1],)))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))
model.compile(optimizer='adam',
          loss='sparse_categorical_crossentropy',
          metrics=['accuracy'])
history = model.fit(x_train, y_train,
                epochs=20,
                batch_size=128)
test_loss, test_acc = model.evaluate(x_test,y_test)
print(test_acc)

Epoch 1/20
50/50 [==============================] - 0s 1ms/step - loss: 1.7609 - accuracy: 0.3653
Epoch 2/20
50/50 [==============================] - 0s 1ms/step - loss: 1.5095 - accuracy: 0.4680
Epoch 3/20
50/50 [==============================] - 0s 1ms/step - loss: 1.4289 - accuracy: 0.4959
Epoch 4/20
50/50 [==============================] - 0s 1ms/step - loss: 1.3693 - accuracy: 0.5107
Epoch 5/20
50/50 [==============================] - 0s 1ms/step - loss: 1.3250 - accuracy: 0.5252
Epoch 6/20
50/50 [==============================] - 0s 1ms/step - loss: 1.2806 - accuracy: 0.5539
Epoch 7/20
50/50 [==============================] - 0s 1ms/step - loss: 1.2366 - accuracy: 0.5648
Epoch 8/20
50/50 [==============================] - 0s 1ms/step - loss: 1.1902 - accuracy: 0.5842
Epoch 9/20
50/50 [==============================] - 0s 1ms/step - loss: 1.1519 - accuracy: 0.6020
Epoch 10/20
50/50 [==============================] - 0s 1ms/step - loss: 1.1059 - accuracy: 0.6229
Epoch 11/20
50/50 [

### Testing Combination of Features

In [54]:
# We use Keras to train and predict genre based on different features combination and try to find out which combination works better
from keras import models
from keras import layers
def cal_accuracy_by_selected_features(features):
    y_lab = np.array(df['genre_top'],dtype=str) #labels
    le = pp.LabelEncoder()
    le.fit(y_lab)
    y = le.transform(y_lab)
    X = np.zeros((7997,1))
    
    for feature in features:
        if feature == 'mfccs_mean' or feature == 'mfccs_var': 
            f = np.nan_to_num(np.stack(df[feature]))
            X = np.append(X,f,1)
        else:
            f = np.zeros((len(X),1))
            f[:,0] = np.array(df[feature])
            X = np.append(X,f,1)
    X = pp.scale(X) 

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)
    model = models.Sequential()
    model.add(layers.Dense(256, activation='relu', input_shape=(X_train.shape[1],)))
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(10, activation='softmax'))
    model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
    history = model.fit(X_train, y_train,
                    epochs=20,
                    batch_size=128)
    test_loss, test_acc = model.evaluate(X_test,y_test)
 
    return test_acc

In [55]:
import itertools

all_features = ['zcr_mean','zcr_var','spec_cent_mean','spec_cent_var','spec_roll_mean','spec_roll_var','mfccs_mean','mfccs_var']

all_comb = []

for length in range(1, len(all_features)):
    for i in itertools.combinations(all_features, length):
        all_comb += [list(i)]

In [56]:
result = []
for features in all_comb:
    accu = cal_accuracy_by_selected_features(features)
    result += [[features, accu]]

Epoch 1/20
32/32 [==============================] - 0s 1ms/step - loss: 2.2014 - accuracy: 0.1521
Epoch 2/20
32/32 [==============================] - 0s 1ms/step - loss: 2.0535 - accuracy: 0.1766
Epoch 3/20
32/32 [==============================] - 0s 1ms/step - loss: 2.0180 - accuracy: 0.1928
Epoch 4/20
32/32 [==============================] - 0s 1ms/step - loss: 2.0121 - accuracy: 0.1971
Epoch 5/20
32/32 [==============================] - 0s 1ms/step - loss: 2.0083 - accuracy: 0.1961
Epoch 6/20
32/32 [==============================] - 0s 1ms/step - loss: 2.0068 - accuracy: 0.2121
Epoch 7/20
32/32 [==============================] - 0s 1ms/step - loss: 2.0013 - accuracy: 0.2129
Epoch 8/20
32/32 [==============================] - 0s 1ms/step - loss: 1.9986 - accuracy: 0.2101
Epoch 9/20
32/32 [==============================] - 0s 1ms/step - loss: 2.0015 - accuracy: 0.2044
Epoch 10/20
32/32 [==============================] - 0s 1ms/step - loss: 1.9982 - accuracy: 0.2114
Epoch 11/20
32/32 [

125/125 [==============================] - 0s 866us/step - loss: 1.9274 - accuracy: 0.2491
Epoch 1/20
32/32 [==============================] - 0s 1ms/step - loss: 2.1402 - accuracy: 0.1816
Epoch 2/20
32/32 [==============================] - 0s 1ms/step - loss: 1.9746 - accuracy: 0.2071
Epoch 3/20
32/32 [==============================] - 0s 1ms/step - loss: 1.9474 - accuracy: 0.2131
Epoch 4/20
32/32 [==============================] - 0s 1ms/step - loss: 1.9388 - accuracy: 0.2264
Epoch 5/20
32/32 [==============================] - 0s 1ms/step - loss: 1.9316 - accuracy: 0.2306
Epoch 6/20
32/32 [==============================] - 0s 1ms/step - loss: 1.9267 - accuracy: 0.2349
Epoch 7/20
32/32 [==============================] - 0s 1ms/step - loss: 1.9272 - accuracy: 0.2349
Epoch 8/20
32/32 [==============================] - 0s 1ms/step - loss: 1.9250 - accuracy: 0.2354
Epoch 9/20
32/32 [==============================] - 0s 1ms/step - loss: 1.9225 - accuracy: 0.2344
Epoch 10/20
32/32 [========

125/125 [==============================] - 0s 794us/step - loss: 1.8340 - accuracy: 0.3376
Epoch 1/20
32/32 [==============================] - 0s 1ms/step - loss: 2.1548 - accuracy: 0.2106
Epoch 2/20
32/32 [==============================] - 0s 1ms/step - loss: 1.9470 - accuracy: 0.2494
Epoch 3/20
32/32 [==============================] - 0s 1ms/step - loss: 1.9105 - accuracy: 0.2706
Epoch 4/20
32/32 [==============================] - 0s 1ms/step - loss: 1.8968 - accuracy: 0.2694
Epoch 5/20
32/32 [==============================] - 0s 1ms/step - loss: 1.8890 - accuracy: 0.2726
Epoch 6/20
32/32 [==============================] - 0s 1ms/step - loss: 1.8827 - accuracy: 0.2751
Epoch 7/20
32/32 [==============================] - 0s 1ms/step - loss: 1.8791 - accuracy: 0.2761
Epoch 8/20
32/32 [==============================] - 0s 1ms/step - loss: 1.8765 - accuracy: 0.2739
Epoch 9/20
32/32 [==============================] - 0s 1ms/step - loss: 1.8777 - accuracy: 0.2799
Epoch 10/20
32/32 [========

32/32 [==============================] - 0s 2ms/step - loss: 1.8317 - accuracy: 0.3054
Epoch 20/20
125/125 [==============================] - 0s 887us/step - loss: 1.8381 - accuracy: 0.3118
Epoch 1/20
32/32 [==============================] - 0s 2ms/step - loss: 2.0936 - accuracy: 0.2141
Epoch 2/20
32/32 [==============================] - 0s 2ms/step - loss: 1.9275 - accuracy: 0.2661
Epoch 3/20
32/32 [==============================] - 0s 2ms/step - loss: 1.9087 - accuracy: 0.2696
Epoch 4/20
32/32 [==============================] - 0s 2ms/step - loss: 1.8947 - accuracy: 0.2776
Epoch 5/20
32/32 [==============================] - 0s 2ms/step - loss: 1.8912 - accuracy: 0.2821
Epoch 6/20
32/32 [==============================] - 0s 2ms/step - loss: 1.8898 - accuracy: 0.2789
Epoch 7/20
32/32 [==============================] - 0s 2ms/step - loss: 1.8802 - accuracy: 0.2794
Epoch 8/20
32/32 [==============================] - 0s 2ms/step - loss: 1.8773 - accuracy: 0.2824
Epoch 9/20
32/32 [========

32/32 [==============================] - 0s 2ms/step - loss: 1.8456 - accuracy: 0.2941
Epoch 19/20
32/32 [==============================] - 0s 2ms/step - loss: 1.8434 - accuracy: 0.2996
Epoch 20/20
125/125 [==============================] - 0s 1ms/step - loss: 1.8583 - accuracy: 0.2976
Epoch 1/20
32/32 [==============================] - 0s 1ms/step - loss: 2.1542 - accuracy: 0.1856
Epoch 2/20
32/32 [==============================] - 0s 1ms/step - loss: 1.9650 - accuracy: 0.2151
Epoch 3/20
32/32 [==============================] - 0s 1ms/step - loss: 1.9372 - accuracy: 0.2376
Epoch 4/20
32/32 [==============================] - 0s 1ms/step - loss: 1.9248 - accuracy: 0.2444
Epoch 5/20
32/32 [==============================] - 0s 1ms/step - loss: 1.9159 - accuracy: 0.2441
Epoch 6/20
32/32 [==============================] - 0s 1ms/step - loss: 1.9043 - accuracy: 0.2591
Epoch 7/20
32/32 [==============================] - 0s 1ms/step - loss: 1.9044 - accuracy: 0.2586
Epoch 8/20
32/32 [=========

...

32/32 [==============================] - 0s 1ms/step - loss: 1.4070 - accuracy: 0.4912
Epoch 5/20
32/32 [==============================] - 0s 1ms/step - loss: 1.3583 - accuracy: 0.5218
Epoch 6/20
32/32 [==============================] - 0s 1ms/step - loss: 1.3155 - accuracy: 0.5368
Epoch 7/20
32/32 [==============================] - 0s 1ms/step - loss: 1.2755 - accuracy: 0.5480
Epoch 8/20
32/32 [==============================] - 0s 1ms/step - loss: 1.2274 - accuracy: 0.5715
Epoch 9/20
32/32 [==============================] - 0s 1ms/step - loss: 1.1955 - accuracy: 0.5840
Epoch 10/20
32/32 [==============================] - 0s 1ms/step - loss: 1.1472 - accuracy: 0.5940
Epoch 11/20
32/32 [==============================] - 0s 1ms/step - loss: 1.1090 - accuracy: 0.6103
Epoch 12/20
32/32 [==============================] - 0s 1ms/step - loss: 1.0727 - accuracy: 0.6281
Epoch 13/20
32/32 [==============================] - 0s 1ms/step - loss: 1.0209 - accuracy: 0.6521
Epoch 14/20
32/32 [========

32/32 [==============================] - 0s 1ms/step - loss: 1.4764 - accuracy: 0.4702
Epoch 4/20
32/32 [==============================] - 0s 1ms/step - loss: 1.4086 - accuracy: 0.4975
Epoch 5/20
32/32 [==============================] - 0s 1ms/step - loss: 1.3648 - accuracy: 0.5128
Epoch 6/20
32/32 [==============================] - 0s 1ms/step - loss: 1.3218 - accuracy: 0.5315
Epoch 7/20
32/32 [==============================] - 0s 1ms/step - loss: 1.2738 - accuracy: 0.5533
Epoch 8/20
32/32 [==============================] - 0s 1ms/step - loss: 1.2288 - accuracy: 0.5748
Epoch 9/20
32/32 [==============================] - 0s 1ms/step - loss: 1.2040 - accuracy: 0.5813
Epoch 10/20
32/32 [==============================] - 0s 1ms/step - loss: 1.1500 - accuracy: 0.5975
Epoch 11/20
32/32 [==============================] - 0s 1ms/step - loss: 1.1116 - accuracy: 0.6141
Epoch 12/20
32/32 [==============================] - 0s 1ms/step - loss: 1.0790 - accuracy: 0.6246
Epoch 13/20
32/32 [=========

32/32 [==============================] - 0s 1ms/step - loss: 1.5917 - accuracy: 0.4320
Epoch 3/20
32/32 [==============================] - 0s 1ms/step - loss: 1.4943 - accuracy: 0.4700
Epoch 4/20
32/32 [==============================] - 0s 1ms/step - loss: 1.4306 - accuracy: 0.4907
Epoch 5/20
32/32 [==============================] - 0s 1ms/step - loss: 1.3797 - accuracy: 0.5175
Epoch 6/20
32/32 [==============================] - 0s 1ms/step - loss: 1.3280 - accuracy: 0.5355
Epoch 7/20
32/32 [==============================] - 0s 1ms/step - loss: 1.2917 - accuracy: 0.5500
Epoch 8/20
32/32 [==============================] - 0s 1ms/step - loss: 1.2440 - accuracy: 0.5690
Epoch 9/20
32/32 [==============================] - 0s 1ms/step - loss: 1.2097 - accuracy: 0.5750
Epoch 10/20
32/32 [==============================] - 0s 1ms/step - loss: 1.1641 - accuracy: 0.6006
Epoch 11/20
32/32 [==============================] - 0s 1ms/step - loss: 1.1162 - accuracy: 0.6191
Epoch 12/20
32/32 [==========

32/32 [==============================] - 0s 3ms/step - loss: 1.9325 - accuracy: 0.3144
Epoch 2/20
32/32 [==============================] - 0s 2ms/step - loss: 1.5890 - accuracy: 0.4347
Epoch 3/20
32/32 [==============================] - 0s 2ms/step - loss: 1.4878 - accuracy: 0.4757
Epoch 4/20
32/32 [==============================] - 0s 2ms/step - loss: 1.4217 - accuracy: 0.5003
Epoch 5/20
32/32 [==============================] - 0s 2ms/step - loss: 1.3765 - accuracy: 0.5103
Epoch 6/20
32/32 [==============================] - 0s 2ms/step - loss: 1.3265 - accuracy: 0.5323
Epoch 7/20
32/32 [==============================] - 0s 2ms/step - loss: 1.2847 - accuracy: 0.5468
Epoch 8/20
32/32 [==============================] - 0s 1ms/step - loss: 1.2421 - accuracy: 0.5655
Epoch 9/20
32/32 [==============================] - 0s 1ms/step - loss: 1.1956 - accuracy: 0.5838
Epoch 10/20
32/32 [==============================] - 0s 1ms/step - loss: 1.1478 - accuracy: 0.6001
Epoch 11/20
32/32 [===========

125/125 [==============================] - 0s 614us/step - loss: 1.7055 - accuracy: 0.4834
Epoch 1/20
32/32 [==============================] - 0s 1ms/step - loss: 1.8958 - accuracy: 0.3259
Epoch 2/20
32/32 [==============================] - 0s 1ms/step - loss: 1.5762 - accuracy: 0.4362
Epoch 3/20
32/32 [==============================] - 0s 1ms/step - loss: 1.4854 - accuracy: 0.4665
Epoch 4/20
32/32 [==============================] - 0s 1ms/step - loss: 1.4358 - accuracy: 0.4875
Epoch 5/20
32/32 [==============================] - 0s 1ms/step - loss: 1.3749 - accuracy: 0.5133
Epoch 6/20
32/32 [==============================] - 0s 1ms/step - loss: 1.3361 - accuracy: 0.5263
Epoch 7/20
32/32 [==============================] - 0s 1ms/step - loss: 1.2848 - accuracy: 0.5533
Epoch 8/20
32/32 [==============================] - 0s 1ms/step - loss: 1.2458 - accuracy: 0.5645
Epoch 9/20
32/32 [==============================] - 0s 1ms/step - loss: 1.2029 - accuracy: 0.5858
Epoch 10/20
32/32 [========

In [57]:
result.sort(key=lambda x: -x[1])
for r in result:
    print(r[0])
    print(r[1])

['zcr_mean', 'spec_cent_mean', 'spec_cent_var', 'mfccs_mean', 'mfccs_var']
0.48412102460861206
['zcr_mean', 'spec_cent_mean', 'spec_cent_var', 'spec_roll_mean', 'spec_roll_var', 'mfccs_mean', 'mfccs_var']
0.483370840549469
['zcr_mean', 'zcr_var', 'spec_cent_var', 'spec_roll_mean', 'spec_roll_var', 'mfccs_mean', 'mfccs_var']
0.48187047243118286
['spec_cent_mean', 'spec_cent_var', 'mfccs_mean', 'mfccs_var']
0.4793698489665985
['zcr_mean', 'zcr_var', 'spec_roll_var', 'mfccs_mean', 'mfccs_var']
0.4788697063922882
['spec_cent_mean', 'spec_roll_mean', 'spec_roll_var', 'mfccs_mean', 'mfccs_var']
0.4788697063922882
['zcr_mean', 'spec_cent_mean', 'spec_roll_mean', 'spec_roll_var', 'mfccs_mean', 'mfccs_var']
0.4783695936203003
['zcr_mean', 'zcr_var', 'spec_roll_mean', 'mfccs_mean', 'mfccs_var']
0.4778694808483124
['zcr_mean', 'zcr_var', 'spec_cent_var', 'spec_roll_mean', 'mfccs_mean', 'mfccs_var']
0.4778694808483124
['zcr_var', 'spec_cent_mean', 'spec_roll_mean', 'spec_roll_var', 'mfccs_mean', '